In [1]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import requests

In [2]:
data = pd.read_csv('abalone_dataset.csv')
# assuming 'df' is your DataFrame
dummies = pd.get_dummies(data['sex'])

# rename the columns
dummies.columns = ['male', 'female', 'other']

data = pd.concat([data, dummies], axis=1)

# drop the original 'sex' column
data = data.drop('sex', axis=1)
data = data.drop('male', axis=1)
data = data.drop('female', axis=1)
# data['type'] = data['type'].replace(3, 2)

# feature_cols = ['sex','length','diameter','height','whole_weight','shucked_weight','viscera_weight','shell_weight','type']
# feature_cols = ['sex','shucked_weight','shell_weight','type']
# data = data[feature_cols]
y = data.type
X = data.drop(['type'], axis=1)

In [9]:
# Train the first model to predict class 1 vs not class 1
clf1 = RandomForestClassifier()
clf2 = RandomForestClassifier()

y_train_1 = y.replace(3, 2)
clf1.fit(X, y_train_1)

aux = pd.concat([X, y], axis=1)
mask = aux['type'] != 1
data_23 = aux[mask]

y_train_23 = data_23.type
X_train_23 = data_23.drop(['type'], axis=1)

clf2.fit(X_train_23, y_train_23)

RandomForestClassifier()

In [4]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# # Train the first model to predict class 1 vs not class 1
# y_train_1 = y_train.replace(3, 2)
# clf1 = RandomForestClassifier()
# clf1.fit(X_train, y_train_1)

# # Train the second model to predict class 2 vs class 3
# aux = pd.concat([X_train, y_train], axis=1)
# mask = aux['type'] != 1
# data_23 = aux[mask]
# y_train_23 = data_23.type
# X_train_23 = data_23.drop(['type'], axis=1)

# clf2 = RandomForestClassifier()
# clf2.fit(X_train_23, y_train_23)

# # Make predictions on the test set
# y_pred_1 = clf1.predict(X_test)

# X_test['type'] = y_pred_1
# aux = X_test
# mask = aux['type'] != 1
# data_23 = aux[mask]
# X_test_23 = data_23.drop(['type'], axis=1)
# X_test = X_test.drop(['type'], axis=1)

# y_pred_23 = clf2.predict(X_test_23)
# y_pred = y_pred_1.copy()
# y_pred[mask] = y_pred_23

# # Evaluate the model performance
# from sklearn.metrics import classification_report
# print(classification_report(y_test, y_pred))


In [5]:
def prediction(dataset):
    y_pred1 = clf1.predict(dataset)
    y_pred = []
    for i in range(len(y_pred1)):
        if y_pred1[i] == 1:
            y_pred.append((y_pred1[i]))
        else:
            value = (dataset.iloc[i].values)[np.newaxis, :]
            prediction = clf2.predict(value)
            y_pred.append((prediction)[0])
    return y_pred

In [6]:
data_app = pd.read_csv('abalone_app.csv')
dummies = pd.get_dummies(data_app['sex'])

dummies.columns = ['male', 'female', 'other']

data_app = pd.concat([data_app, dummies], axis=1)

data_app = data_app.drop('sex', axis=1)
data_app = data_app.drop('male', axis=1)
data_app = data_app.drop('female', axis=1)

y_pred = prediction(data_app)


 - Aplicando modelo e enviando para o servidor


c:\Users\Rafae\Documents\Geral\Ufal\P8\ML\03_Validation\venv\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\Rafae\Documents\Geral\Ufal\P8\ML\03_Validation\venv\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\Rafae\Documents\Geral\Ufal\P8\ML\03_Validation\venv\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\Rafae\Documents\Geral\Ufal\P8\ML\03_Validation\venv\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\Rafae\Documents\Geral\Ufal\P8\ML\03_Validation\venv\lib\site-packages\sklearn\base.py:439: User

In [8]:
# Enviando previsões realizadas com o modelo para o servidor
URL = "https://aydanomachado.com/mlclass/03_Validation.php"

#TODO Substituir pela sua chave aqui
DEV_KEY = "AiDANO"

# json para ser enviado para o servidor
data = {'dev_key':DEV_KEY,
        'predictions':pd.Series(y_pred).to_json(orient='values')}

# Enviando requisição e salvando o objeto resposta
r = requests.post(url = URL, data = data)

# Extraindo e imprimindo o texto da resposta
pastebin_url = r.text
print(" - Resposta do servidor:\n", r.text, "\n")

 - Resposta do servidor:
 {"error":{"code":102,"message":"Espere ao menos 12 horas entre dois envios, tempo restante 00 dias 11 horas 49 minutos 10 segundos"}} 

